### Retrieve from MountainHub

This code allow for direct access to the MountainHub API. Data are returned in a Pandas dataframe.

Some code is adopted from Don Setiawan's work stored in the [cso legacy api](https://github.com/communitysnowobs/cso-legacy-api).

A. Arendt 20191231


In [1]:
import sys
import time
import datetime

import pandas as pd
import geopandas as gpd
import requests

BASE_URL = 'https://api.mountainhub.com/timeline'
HEADER = { 'Accept-version': '1' }
ONE_MONTH = 2592000000

Load these [utility functions from the cso-legacy-api](https://github.com/communitysnowobs/cso-legacy-api/blob/master/src/common/utils.py).

Todo: move these to a script after testing.

In [29]:
def empty_cso_dataframe():
    return pd.DataFrame(columns = [
        'author_name',
        'id',
        'timestamp',
        'lat',
        'long',
        'snow_depth',
        'source',
        'elevation',
        'comment'
    ])

def date_to_timestamp(date):
    """Converts datetime object to unix timestamp.
    Keyword arguments:
    date -- Datetime object to convert
    """
    if date is None:
        return date
    return int(time.mktime(date.timetuple())) * 1000

def timestamp_to_date(timestamp):
    """Converts unix timestamp to datettime object.
    Keyword arguments:
    timestamp -- Timestamp to convert
    """
    if timestamp is None:
        return timestamp
    return datetime.date.fromtimestamp(timestamp / 1000)

In [3]:
now_ts = date_to_timestamp(datetime.datetime.now())
start_ts = date_to_timestamp(datetime.datetime(2019,12,31))

Load parsing function from the [MountainHub cso-legacy-api script](https://github.com/communitysnowobs/cso-webapp/blob/dev/src/csoapi/apps/cso/types/snowobs.py).

#### NOTE: the MountainHub API has different named fields than what we were previously using in the legacy script above.

The code below is an attempt to match what is now in the MountainHub API:

In [4]:
def parse_data(results):
    observations = []

    for idx, res in enumerate(results):
        obs_data = {}
        try:
            observation = res['observation']
            obs_data['obs_id'] = observation['_id']
            obs_data['timestamp'] = int(observation['reported_at'])
            obs_data['obs_type'] = observation['type']
            obs_data['comment'] = observation['description']
            if len(observation['details']) > 0:
                if observation['details'][0]:
                    if 'snowpack_depth' in observation['details'][0].keys():
                        obs_data['snow_depth'] = observation['details'][0]['snowpack_depth'] 
        except:    
            obs_data['obs_id'] = 'None'
            #obs_data['timestamp'] = 'None'
            obs_data['obs_type'] = 'None'
            #obs_data['snow_depth'] = some dummy value?
        finally:
            actor = res['actor']
            if 'full_name' in actor.keys():
                obs_data['author_name'] = actor['full_name']
            elif 'fullName' in actor.keys():
                obs_data['author_name'] = actor['fullName']
            obs_data['id'] = idx
            obs_data['lat'] = res['location']['coordinates'][1]
            obs_data['lng'] = res['location']['coordinates'][0]
            obs_data['source'] = 'MountainHub'
        observations.append(obs_data)

    df = pd.DataFrame.from_records(observations).dropna()

    return df[df['snow_depth'] != 'undefined']

In [5]:
def fetch_raw_data(min_timestamp, max_timestamp, is_raw_json=False):

    args = {
      'publisher': 'all',
      'limit': 10000,
      'since' : min_timestamp,
      'before' : max_timestamp,
    }

    response = requests.get(BASE_URL, params=args, headers=HEADER)
    data = response.json()
    results = data['results']
    return parse_data(results)

In [9]:
df = fetch_raw_data(start_ts, now_ts )

In [46]:
df.columns

Index(['obs_id', 'obs_type', 'author_name', 'id', 'lat', 'lng', 'source',
       'timestamp', 'comment', 'snow_depth'],
      dtype='object')

In [49]:
df.assign(date = df['timestamp'].apply(timestamp_to_date)).drop(columns = ['timestamp','obs_id']).to_csv('data.csv')